# Importação das bibliotecas

In [4]:
import pandas as pd

# Gerador de gráficos
import plotly.express as px

# Limpeza de warnings 
import warnings
warnings.simplefilter('ignore')

# Carregamento e Limpeza dos dados

In [5]:
gastos_carregados = pd.read_csv(
    'https://www.gov.br/secretariageral/pt-br/acesso-a-informacao/informacoes-classificadas-e-desclassificadas/Planilha12003a2022.csv',
    encoding='latin1', delimiter=';')
ipca_carregado = pd.read_csv('Arquivos_suporte/IPCA mensal 1994-2022.csv',
    encoding='utf-8', delimiter=';')

In [6]:
gastos = gastos_carregados.copy()
ipca = ipca_carregado.copy()

### Manipulação do dataframe de gastos

In [7]:
# Muda o formato da data para datetime
gastos.loc[:, 'DATA PGTO'] = pd.to_datetime(gastos['DATA PGTO'], errors='coerce', format="%d/%m/%Y")

# Gera uma nova coluna com os anos
gastos.loc[:, 'ANO'] = gastos['DATA PGTO'].dt.year

# Remove os dados faltantes de acordo com a coluna 'CPF SERVIDOR'
gastos = gastos.dropna(axis=0, subset=['CPF SERVIDOR'])

# Cria uma coluna nova com os valores em Reais no formato float
gastos.loc[:, 'VALOR_NUM'] = pd.to_numeric(gastos['VALOR'].str.replace(
    r'R\$', '').str.replace(".", '').str.replace(',', '.').str.strip(), errors='coerce')

# Muda o formato dos dados de tipo(modalidade de pagamento):
gastos.loc[:, "TIPO"] = gastos.loc[:, "TIPO"].replace('D','Débito').replace('C', 'Crédito')

# Remove as dados faltantes que não tenha nenhum valor de gastos
gastos = gastos.dropna(subset=['VALOR_NUM'])

In [8]:
# Converte o ano para o nome do presidente
def presidente(ano):
    if ano <= 2010:
        return 'LULA'
    elif ano <= 2016:
        return 'DILMA'
    elif ano <= 2018:
        return "TEMER"
    elif ano <= 2022:
        return "BOLSONARO"

# Gera uma coluna com a informação do nome do atual presidente
gastos['PRES'] = gastos['ANO'].apply(presidente)

### Manipulação do dataframe do ipca

In [9]:
ipca = ipca.T.reset_index()
ipca.columns = ['DATA', 'INFLAÇÃO NO MÊS']

In [10]:
# Limpeza dos dados: converte o mês de março para 'mar' assim como os outros meses estão abreviados
ipca['DATA'] = ipca['DATA'].str.replace('março ' ,'mar/')

# Converte os anos para números
ipca['DATA'] = ipca['DATA'].str.replace('/199' ,'/9')
ipca['DATA'] = ipca['DATA'].str.replace('/200' ,'/0')
ipca['DATA'] = ipca['DATA'].str.replace('/201' ,'/1')
ipca['DATA'] = ipca['DATA'].str.replace('/202' ,'/2')

ipca[['MES','ANO']] = ipca['DATA'].str.split('/',expand=True)

# Converte os meses para numeros
mes_para_numero = {'jan':1, 'fev':2, 'mar':3, 'abr':4, 'mai':5, 'jun':6, 'jul':7, 'ago':8, 'set':9, 'out':10, 'nov':11, 'dez':12}
ipca['MES_NUM'] = ipca['MES'].map(mes_para_numero)

In [11]:
ipca['ANO'] = pd.to_numeric(ipca['ANO'], errors='coerce')

def formata_ano(ano):
    if ano <23:
        return ano + 2000
    if ano >23:
        return ano + 1900
    
ipca['ANO_FORMATADO'] = ipca['ANO'].apply(formata_ano)
ipca['MES-ANO'] = ipca['MES_NUM'].astype(str) + ipca['ANO_FORMATADO'].astype(str)
ipca = ipca.drop(['MES', 'ANO', 'MES_NUM', 'ANO_FORMATADO'], axis=1)

In [12]:
indice_corte = ipca.loc[ipca['MES-ANO'] == '122002'].index[0]
ipca = ipca[indice_corte+1:]
ipca['INFLAÇÃO NO MÊS'] = ipca['INFLAÇÃO NO MÊS']/100

In [13]:
ipca = ipca.iloc[::-1]
ipca['AJUSTE INFLAÇÃO'] = (ipca['INFLAÇÃO NO MÊS']+1).cumprod()

### Merge dataframe gastos - ipca

In [14]:
# Converte o campo mês para um valor de tipo mensal
gastos.loc[:, 'MES'] = gastos['DATA PGTO'].dt.month

gastos['MES-ANO'] = gastos['MES'].astype(str) + gastos['ANO'].astype(str)
gastos['MES-ANO'] = gastos['MES-ANO'].astype(str)

gastos['MES-ANO'] = pd.to_numeric(gastos['MES-ANO'], errors='coerce')
ipca['MES-ANO'] = pd.to_numeric(ipca['MES-ANO'], errors='coerce')
gastos = pd.merge(gastos, ipca, how='outer', on='MES-ANO')

In [15]:
# Efetua a multiplicação dos campos de cada valor mensal pela inflação para corigir o valor gasto por mês
gastos['VALOR_CORRIGIDO'] = gastos['VALOR_NUM']*gastos['AJUSTE INFLAÇÃO']

In [16]:
gastos

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ANO,VALOR_NUM,PRES,MES,MES-ANO,DATA,INFLAÇÃO NO MÊS,AJUSTE INFLAÇÃO,VALOR_CORRIGIDO
0,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",Débito,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1696.90,LULA,1,12003.0,jan/03,0.0225,3.173955,5385.884824
1,2003-01-02,***.010.008-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",Débito,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1336.60,LULA,1,12003.0,jan/03,0.0225,3.173955,4242.308714
2,2003-01-02,***.868.251-**,3.134920e+13,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",Débito,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,2003.0,1396.43,LULA,1,12003.0,jan/03,0.0225,3.173955,4432.206462
3,2003-01-03,***.004.131-**,5.927170e+11,FLORES ALVORADA,"R$ 8.585,00",Crédito,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,8585.00,LULA,1,12003.0,jan/03,0.0225,3.173955,27248.406634
4,2003-01-03,***.004.131-**,8.393080e+11,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",Crédito,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,2003.0,251.50,LULA,1,12003.0,jan/03,0.0225,3.173955,798.249769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113319,2022-12-03,***.136.591-**,1.726166e+13,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",Crédito,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,179.50,BOLSONARO,12,122022.0,dez/22,0.0062,1.006200,180.612900
113320,2022-12-03,***.136.591-**,3.987994e+13,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",Crédito,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,54.95,BOLSONARO,12,122022.0,dez/22,0.0062,1.006200,55.290690
113321,2022-12-04,***.136.591-**,4.750841e+13,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",Crédito,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,518.07,BOLSONARO,12,122022.0,dez/22,0.0062,1.006200,521.282034
113322,2022-12-04,***.136.591-**,1.255044e+13,RESTAURANTE SOHO,"330,52",Crédito,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,2022.0,330.52,BOLSONARO,12,122022.0,dez/22,0.0062,1.006200,332.569224


### Dataframe de gastos por presidente acumulado por ano

In [17]:
# Gastos do presidente por ano
gastos_presidente = gastos.groupby(["ANO", "PRES"]).agg({"VALOR_NUM": sum, "VALOR_CORRIGIDO": sum}).reset_index()
gastos_presidente

,ANO,PRES,VALOR_NUM,VALOR_CORRIGIDO
0,2003.0,LULA,5327599.63,1.594562e+07
1,2004.0,LULA,6541238.37,1.839041e+07
2,2005.0,LULA,5204035.80,1.362267e+07
3,2006.0,LULA,4977941.73,1.251524e+07
4,2007.0,LULA,3857905.40,9.373571e+06
5,2008.0,LULA,6084616.50,1.391479e+07
6,2009.0,LULA,5438905.30,1.192018e+07
7,2010.0,LULA,6557394.19,1.366002e+07
8,2011.0,DILMA,4587842.43,8.976243e+06
9,2012.0,DILMA,4598570.59,8.542153e+06


In [18]:
fig = px.bar(gastos_presidente, x='ANO', y="VALOR_NUM", color='PRES',
             title="Gastos do cartão presidencial por presidente",
             labels={"VALOR_NUM": "Valor em milhões", 'PRES': "PRESIDENTE"})

fig = fig.update_xaxes(tickangle=45, tickvals=gastos_presidente['ANO'][::])
fig.show()

### Gastos Corrigidos pelo IPCA

In [19]:
fig = px.bar(gastos_presidente, x='ANO', y="VALOR_CORRIGIDO", color='PRES',
             title="Gastos do cartão presidencial por presidente",
             labels={"VALOR_CORRIGIDO": "Valor em milhões (corrigido pelo IPCA)", 'PRES': "PRESIDENTE"})

fig = fig.update_xaxes(tickangle=45, tickvals=gastos_presidente['ANO'][::])
fig.show()

In [20]:
# Tabela por andato presidencial

lula_mandato_1 = gastos_presidente.loc[0:3].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
lula_mandato_1['PRES'] = 'LULA_1'

lula_mandato_2 = gastos_presidente.loc[4:7].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
lula_mandato_2['PRES'] = 'LULA_2'

dilma_mandato_1 = gastos_presidente.loc[8:11].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
dilma_mandato_1['PRES'] = 'DILMA_1'

dilma_mandato_2 = gastos_presidente.loc[12:13].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
dilma_mandato_2['PRES'] = 'DILMA_2'

temer = gastos_presidente.loc[14:15].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
bolsonaro = gastos_presidente.loc[16:].groupby(["PRES"]).agg({"VALOR_NUM": sum}).reset_index()
bolsonaro

gastos_acumulados = pd.concat([lula_mandato_1, lula_mandato_2, dilma_mandato_1, dilma_mandato_2, temer, bolsonaro], axis=0, ignore_index=True)
gastos_acumulados

,PRES,VALOR_NUM
0,LULA_1,22050815.53
1,LULA_2,21938821.39
2,DILMA_1,24513516.26
3,DILMA_2,9899205.36
4,TEMER,8869681.44
5,BOLSONARO,27621657.23


In [21]:
#Gráfico por mandato presidencial
fig_acumulado = px.bar(gastos_acumulados, x='PRES', y='VALOR_NUM', color='PRES', title='Gasto acumulado do cartão presidencial por mandato',
labels={"VALOR_NUM": "Valor em milhões    *sem correção da inflação", 'PRES': "Presidente"})

fig_acumulado.show()

In [22]:
#tabela por mandato presidencial com valor corrigido pela inflação
lula_mandato_1 = gastos_presidente.loc[0:3].groupby(["PRES"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
lula_mandato_1['PRES'] = 'LULA_1'

lula_mandato_2 = gastos_presidente.loc[4:7].groupby(["PRES"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
lula_mandato_2['PRES'] = 'LULA_2'

dilma_mandato_1 = gastos_presidente.loc[8:11].groupby(["PRES"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
dilma_mandato_1['PRES'] = 'DILMA_1'

dilma_mandato_2 = gastos_presidente.loc[12:13].groupby(["PRES"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
dilma_mandato_2['PRES'] = 'DILMA_2'

temer = gastos_presidente.loc[14:15].groupby(["PRES"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
bolsonaro = gastos_presidente.loc[16:].groupby(["PRES"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
bolsonaro

gastos_acumulados_corrigidos = pd.concat([lula_mandato_1, lula_mandato_2, dilma_mandato_1, dilma_mandato_2, temer, bolsonaro], axis=0, ignore_index=True)
gastos_acumulados_corrigidos

,PRES,VALOR_CORRIGIDO
0,LULA_1,6.047393e+07
1,LULA_2,4.886856e+07
2,DILMA_1,4.334890e+07
3,DILMA_2,1.445050e+07
4,TEMER,1.165053e+07
5,BOLSONARO,3.175610e+07


In [23]:
fig_acumulado_corrigido = px.bar(gastos_acumulados_corrigidos, x='PRES', y='VALOR_CORRIGIDO', color='PRES', title='Gasto acumulado do cartão presidencial por mandato',
labels={"VALOR_CORRIGIDO": "Valor em milhões *corrigido pelo IPCA", 'PRES': "Presidente"})

fig_acumulado_corrigido.show()

### Análise dos Gastos por setores (Subelementos de Despesa)

In [24]:
gastos_gerais_setores = gastos.groupby(["SUBELEMENTO DE DESPESA"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
gastos_gerais_setores = gastos_gerais_setores.sort_values('VALOR_CORRIGIDO', ascending=False).reset_index()
gastos_gerais_setores

,index,SUBELEMENTO DE DESPESA,VALOR_CORRIGIDO
0,32,HOSPEDAGENS,9.954654e+07
1,35,LOCACAO DE MEIOS DE TRANSPORTE,3.088881e+07
2,31,GENEROS DE ALIMENTAÇO,2.839001e+07
3,28,FORNECIMENTO DE ALIMENTAÇO,2.373515e+07
4,87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",1.286672e+07
...,...,...,...
107,74,MATERIAL P/REABILITAÇO PROFISSIONAL,1.027113e+02
108,99,SERVIOS DE µGUA E ESGOTO,7.805811e+01
109,52,MATERIAL BIOLàGICO,3.779940e+01
110,80,OUTRAS DESPESAS COM LOCOMOÇO,2.335202e+01


#### Despesas com maiores gastos

In [25]:
gastos_gerais_setores.head(10)

,index,SUBELEMENTO DE DESPESA,VALOR_CORRIGIDO
0,32,HOSPEDAGENS,9.954654e+07
1,35,LOCACAO DE MEIOS DE TRANSPORTE,3.088881e+07
2,31,GENEROS DE ALIMENTAÇO,2.839001e+07
3,28,FORNECIMENTO DE ALIMENTAÇO,2.373515e+07
4,87,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",1.286672e+07
5,98,SERVIOS DE TELECOMUNICAåES,2.391531e+06
6,11,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,1.983723e+06
7,59,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,1.904220e+06
8,36,LOCAÇO BENS MOV. OUTR. NATUREZA E INTANGÖVEIS,1.607728e+06
9,56,MATERIAL DE COPA E COZINHA,7.213524e+05


#### Lula

In [26]:
gastos_lula = gastos[gastos['PRES'] == 'LULA']

In [27]:
gastos_lula_setores = gastos_lula.groupby(["TIPO", "SUBELEMENTO DE DESPESA"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
gastos_lula_setores = gastos_lula_setores.sort_values('VALOR_CORRIGIDO', ascending=False).reset_index()
# Os 20 maiores gastos do presidente Lula
gastos_lula_setores.head(20)

,index,TIPO,SUBELEMENTO DE DESPESA,VALOR_CORRIGIDO
0,18,Crédito,HOSPEDAGENS,4.457278e+07
1,20,Crédito,LOCACAO DE MEIOS DE TRANSPORTE,2.182620e+07
2,17,Crédito,GENEROS DE ALIMENTAÇO,9.706771e+06
3,100,Débito,LOCACAO DE MEIOS DE TRANSPORTE,8.832066e+06
4,14,Crédito,FORNECIMENTO DE ALIMENTAÇO,6.109631e+06
5,98,Débito,HOSPEDAGENS,4.501830e+06
6,94,Débito,FORNECIMENTO DE ALIMENTAÇO,2.618778e+06
7,63,Crédito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",1.989016e+06
8,155,Débito,SERVIOS DE TELECOMUNICAåES,1.751158e+06
9,40,Crédito,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,8.174789e+05


In [28]:
fig_lula = px.pie(gastos_lula_setores, values='VALOR_CORRIGIDO', names='TIPO', title='Gastos Lula por modalidade de pagamento',
color_discrete_sequence=px.colors.sequential.RdBu)
fig_lula.show()

In [29]:
fig_lula_setores = px.pie(gastos_lula_setores.head(20), values='VALOR_CORRIGIDO', names='SUBELEMENTO DE DESPESA', 
                          title='Principais Gastos de Lula por Setor',color_discrete_sequence=px.colors.sequential.RdBu)
fig_lula_setores.show()

#### Dilma

In [30]:
gastos_dilma = gastos[gastos['PRES'] == 'DILMA']

gastos_dilma_setores = gastos_dilma.groupby(["TIPO", "SUBELEMENTO DE DESPESA"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
gastos_dilma_setores = gastos_dilma_setores.sort_values('VALOR_CORRIGIDO', ascending=False).reset_index()
gastos_dilma_setores.head(20)

,index,TIPO,SUBELEMENTO DE DESPESA,VALOR_CORRIGIDO
0,21,Crédito,HOSPEDAGENS,3.012915e+07
1,20,Crédito,GENEROS DE ALIMENTAÇO,9.100243e+06
2,17,Crédito,FORNECIMENTO DE ALIMENTAÇO,6.699946e+06
3,64,Crédito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",5.576936e+06
4,127,Débito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",9.324621e+05
5,41,Crédito,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,7.080077e+05
6,134,Débito,SERVIOS DE TELECOMUNICAåES,5.100512e+05
7,48,Crédito,MATERIAL FARMACOLOGICO,4.309409e+05
8,92,Débito,GENEROS DE ALIMENTAÇO,4.144035e+05
9,7,Crédito,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,3.581242e+05


In [31]:
fig_dilma = px.pie(gastos_dilma_setores, values='VALOR_CORRIGIDO', names='TIPO', title='Gastos Dilma por modalidade de pagamento',
color_discrete_sequence=px.colors.sequential.algae)
fig_dilma.show()

In [32]:
fig_dilma_setores = px.pie(gastos_dilma_setores.head(20), values='VALOR_CORRIGIDO', names='SUBELEMENTO DE DESPESA', title='Principais Gastos de Dilma por Setor',
color_discrete_sequence=px.colors.sequential.algae)
fig_dilma_setores.show()

#### Temer

In [33]:
gastos_temer = gastos[gastos['PRES'] == 'TEMER']

gastos_temer_setores = gastos_temer.groupby(["TIPO", "SUBELEMENTO DE DESPESA"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
gastos_temer_setores = gastos_temer_setores.sort_values('VALOR_CORRIGIDO', ascending=False).reset_index()
gastos_temer_setores.head(20)

,index,TIPO,SUBELEMENTO DE DESPESA,VALOR_CORRIGIDO
0,10,Crédito,HOSPEDAGENS,4.233646e+06
1,9,Crédito,GENEROS DE ALIMENTAÇO,3.048765e+06
2,37,Crédito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",1.864819e+06
3,7,Crédito,FORNECIMENTO DE ALIMENTAÇO,1.623661e+06
4,57,Débito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",2.568768e+05
5,23,Crédito,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,8.266688e+04
6,2,Crédito,COMBUSTIVEIS E LUBRIF. AUTOMOTIVOS,8.248911e+04
7,50,Débito,FORNECIMENTO DE ALIMENTAÇO,7.297746e+04
8,52,Débito,HOSPEDAGENS,5.897169e+04
9,58,Débito,SERVIOS DE COMUNICAÇO EM GERAL,5.435830e+04


In [34]:
fig_temer = px.pie(gastos_temer_setores, values='VALOR_CORRIGIDO', names='TIPO', title='Gastos Temer por modalidade de pagamento',
color_discrete_sequence=px.colors.sequential.Blues_r)
fig_temer.show()

In [35]:
fig_temer_setores = px.pie(gastos_temer_setores.head(20), values='VALOR_CORRIGIDO', names='SUBELEMENTO DE DESPESA', title='Principais Gastos de Temer por Setor',
color_discrete_sequence=px.colors.sequential.Blues_r)
fig_temer_setores.show()

#### Bolsonaro

In [36]:
gastos_bolsonaro = gastos[gastos['PRES'] == 'BOLSONARO']

gastos_bolsonaro_setores = gastos_bolsonaro.groupby(["TIPO", "SUBELEMENTO DE DESPESA"]).agg({"VALOR_CORRIGIDO": sum}).reset_index()
gastos_bolsonaro_setores = gastos_bolsonaro_setores.sort_values('VALOR_CORRIGIDO', ascending=False).reset_index()
gastos_bolsonaro_setores

,index,TIPO,SUBELEMENTO DE DESPESA,VALOR_CORRIGIDO
0,14,Crédito,HOSPEDAGENS,1.552674e+07
1,11,Crédito,FORNECIMENTO DE ALIMENTAÇO,6.048891e+06
2,13,Crédito,GENEROS DE ALIMENTAÇO,5.460735e+06
3,73,Débito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",9.705348e+05
4,45,Crédito,"SERV.DE APOIO ADMIN.,TECNICO E OPERACIONAL",8.183413e+05
...,...,...,...,...
74,15,Crédito,LIMPEZA E CONSERVAÇO,1.198054e+02
75,51,Crédito,SERVIOS TECNICOS PROFISSIONAIS,8.727249e+01
76,67,Débito,MAN. E CONSERV. DE MµQUINAS E EQUIPAMENTOS,5.508869e+01
77,38,Crédito,MATERIAL LABORATORIAL,3.331855e+01


In [37]:
fig_bolsonaro = px.pie(gastos_bolsonaro_setores, values='VALOR_CORRIGIDO', names='TIPO', title='Gastos Bolsonaro por modalidade de pagamento',
color_discrete_sequence=px.colors.sequential.YlOrRd)
fig_bolsonaro.show()

In [38]:
fig_Bolsonaro_setores = px.pie(gastos_bolsonaro_setores.head(20), values='VALOR_CORRIGIDO', names='SUBELEMENTO DE DESPESA', title='Principais Gastos Bolsonaro por Setor',
color_discrete_sequence=px.colors.sequential.YlOrRd)
fig_Bolsonaro_setores.show()